In [291]:
import numpy
import plotly.graph_objects as go

In [292]:
V_coll = numpy.array([
    [0, 0, 0],
    [1, 0, 0],
    [0, 1, 0],
], dtype=float)

E_coll = numpy.array([
    [0, 1],
    [1, 2],
    [2, 0]
])

V_fem = numpy.array([
    [0, 0, 0], # 0
    [1, 0, 0], # 1
    [0, 1, 0], # 2
    [0.25, 0, 0], # 3
    [0.5, 0, 0], # 4
    [0.75, 0, 0], # 5
    [0.75, 0.25, 0], # 6
    [0.5, 0.5, 0], # 7
    [0.25, 0.75, 0], # 8
    [0, 0.75, 0], # 9
    [0, 0.5, 0], # 10
    [0, 0.25, 0], # 11
], dtype=float)

E_fem = numpy.array([
    [0, 3],
    [3, 4],
    [4, 5],
    [5, 1],
    [1, 6],
    [6, 7],
    [7, 8],
    [8, 2],
    [2, 9],
    [9, 10],
    [10, 11],
    [11, 0],
])

In [293]:
def upsample(V, E, edge_splits):
    alphas = numpy.linspace(0, 1, edge_splits + 2)[1:-1]
    upsampled_V = [V]
    downsample_buddy = [[] for _ in range(V.shape[0])]
    i= V.shape[0]
    for e in E:
        v0, v1 = V[e]
        for alpha in alphas:
            if alpha <= 0.5:
                downsample_buddy[e[0]].append(i)
            if alpha >= 0.5:
                downsample_buddy[e[1]].append(i)
            upsampled_V.append((v1 - v0) * alpha + v0)
            i += 1
    return numpy.vstack(upsampled_V), downsample_buddy

def downsample(W, downsample_buddies):
    W_downsampled = numpy.vstack([
        W[i] + sum(W[buddies])
        for i, buddies in enumerate(downsample_buddies)])
    W_downsampled /= numpy.linalg.norm(W_downsampled, axis=1)[:, None]
    # W_downsampled /= (W_downsampled != 0).sum(axis=1)[:, None]
    # return (W_downsampled != 0).astype(float)
    return W_downsampled

In [294]:
upsampled_V, downsample_buddy = upsample(V_coll, E_coll, 10)

In [295]:
def point_on_edge(p, e0, e1):
    d = numpy.linalg.norm(numpy.cross(e0 - p, e1 - p))**2 / numpy.linalg.norm(e1 - e0)**2
    return d < 1e-8

def point_edge_coord(p, e0, e1):
    e = e1 - e0
    return (p - e0).dot(e) / numpy.linalg.norm(e)**2

W = numpy.zeros((upsampled_V.shape[0], V_fem.shape[0]))

for i, v in enumerate(upsampled_V):
    for e in E_fem:
        e0, e1 = V_fem[e]
        t = point_edge_coord(v, e0, e1)
        if point_on_edge(v, e0, e1) and 0 <= t <= 1:
            W[i, e[0]] = 1 - t
            W[i, e[1]] = t
            break

assert(numpy.linalg.norm(upsampled_V - W @ V_fem) < 1e-12)

In [296]:
W_new = downsample(W, downsample_buddy)
W_new

array([[0.54706829, 0.        , 0.        , 0.52942092, 0.26471046,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.26471046, 0.52942092],
       [0.        , 0.54706829, 0.        , 0.        , 0.26471046,
        0.52942092, 0.52942092, 0.26471046, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.54706829, 0.        , 0.        ,
        0.        , 0.        , 0.26471046, 0.52942092, 0.52942092,
        0.26471046, 0.        ]])

In [297]:
downsample(W, downsample_buddy)

array([[0.54706829, 0.        , 0.        , 0.52942092, 0.26471046,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.26471046, 0.52942092],
       [0.        , 0.54706829, 0.        , 0.        , 0.26471046,
        0.52942092, 0.52942092, 0.26471046, 0.        , 0.        ,
        0.        , 0.        ],
       [0.        , 0.        , 0.54706829, 0.        , 0.        ,
        0.        , 0.        , 0.26471046, 0.52942092, 0.52942092,
        0.26471046, 0.        ]])

In [298]:
U_fem = numpy.array([
    [0, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
    [0, 0, 0],
])

In [299]:
print(V_coll)
print(V_coll + W_new @ U_fem)
print(V_fem + U_fem)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]
[[0.         0.         0.        ]
 [1.         0.54706829 0.        ]
 [0.         1.         0.        ]]
[[0.   0.   0.  ]
 [1.   1.   0.  ]
 [0.   1.   0.  ]
 [0.25 0.   0.  ]
 [0.5  0.   0.  ]
 [0.75 0.   0.  ]
 [0.75 0.25 0.  ]
 [0.5  0.5  0.  ]
 [0.25 0.75 0.  ]
 [0.   0.75 0.  ]
 [0.   0.5  0.  ]
 [0.   0.25 0.  ]]


In [300]:
X_coll = V_coll + W_new @ U_fem
X_fem = V_fem + U_fem

coll_row_order = numpy.array([0, 1, 2, 0])
mid_row_order = numpy.array([0, 3, 1, 4, 2, 5, 0])
fem_row_order = numpy.array([0, 3, 4, 5, 1, 6, 7, 8, 2, 9, 10, 11, 0])

from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=upsampled_V[:, 0], y=upsampled_V[:, 1], mode="markers", name="upsampled V_coll"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800)

In [301]:
U_coll = numpy.array([
    [0, 0.2, 0],
    [0, 0.2, 0],
    [0, 0, 0],
])

X_coll = V_coll + U_coll
X_fem = V_fem + W_new.T @ U_coll

coll_row_order = numpy.array([0, 1, 2, 0])
mid_row_order = numpy.array([0, 3, 1, 4, 2, 5, 0])
fem_row_order = numpy.array([0, 3, 4, 5, 1, 6, 7, 8, 2, 9, 10, 11, 0])

from plotly.subplots import make_subplots
fig = make_subplots(rows=2, cols=3)

fig.add_trace(go.Scatter(x=V_coll[coll_row_order, 0], y=V_coll[coll_row_order, 1], name="V_coll"), row=1, col=1),
fig.add_trace(go.Scatter(x=upsampled_V[:, 0], y=upsampled_V[:, 1], mode="markers", name="upsampled V_coll"), row=1, col=2),
fig.add_trace(go.Scatter(x=V_fem[fem_row_order, 0], y=V_fem[fem_row_order, 1], name="V_fem"), row=1, col=3),
fig.add_trace(go.Scatter(x=X_coll[coll_row_order, 0], y=X_coll[coll_row_order, 1], name="Deformed V_coll"), row=2, col=1),
fig.add_trace(go.Scatter(x=X_fem[fem_row_order, 0], y=X_fem[fem_row_order, 1], name="Deformed V_fem"), row=2, col=3),

fig.update_layout(width=1200, height=800)
fig.update_xaxes(range=[-0.1, 1.1])
fig.update_yaxes(range=[-0.1, 1.1])